# Version Control


In [ ]:
import sys
package_path = '../input/pytorch-image-models/pytorch-image-models-master'
sys.path.append(package_path)

In [ ]:
import os
import gc
import time
import numpy as np
import pandas as pd
from tqdm import tqdm
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2, ToTensor
import albumentations as alb
import torchvision.models as tv_models

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import cv2
import timm

import warnings
warnings.simplefilter('ignore')

In [ ]:
device = ("cuda" if torch.cuda.is_available() else "cpu")
print("Using %s" % (str(device).upper()))

In [ ]:
def seed_everything(seed):
    print("SEED SET TO : %d" % seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [ ]:
BATCH_SIZE = 32
NUM_SPLITS = 5
IMG_SIZE = (512, 512)
EPOCHS_USED = [1, 2, 3, 4, 5]
TARGET_SIZE = 5
TTA = 1
SEED = 420
TEST_PATH = "../input/cassava-leaf-disease-classification/test_images/"
seed_everything(SEED)

In [ ]:
train_df = pd.read_csv("../input/cassava-leaf-disease-classification/train.csv")
sample_sub = pd.read_csv("../input/cassava-leaf-disease-classification/sample_submission.csv")

In [ ]:
class Cassava_dataset:
    def __init__(self, df, transforms, img_size, img_path, is_train=True):
        self.transforms = transforms
        self.df = df
        self.img_size = img_size
        self.is_train = is_train
        self.img_path = img_path
        
    def __getitem__(self, index):
        img_path = self.img_path + str(self.df["image_id"].loc[index])
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.transforms:
            image = self.transforms(image=image)['image']
            
        if self.is_train:
            label = self.df["label"].loc[index]
            return {
                "image": image,
                "label": torch.tensor(label, dtype=torch.long)
            }
        else:
            return {
                "image": image
            }
        
    def __len__(self):
        return len(self.df)

In [ ]:
class EffNetb1(torch.nn.Module):
    def __init__(self,):
        super(EffNetb1, self).__init__()
        self.model = timm.create_model("tf_efficientnet_b1_ns", pretrained=False)
        n_features = self.model.classifier.in_features
        self.fc = nn.Linear(n_features, TARGET_SIZE)
        self.model.classifier = nn.Identity()
        
    def forward(self, x):
        x = self.model(x)
        x = self.fc(x)
        return x
      
def res_spoon():
    model = timm.create_model("resnext50_32x4d", pretrained=False)
    in_features = model.fc.in_features
    model.fc = nn.Linear(in_features=in_features, out_features=TARGET_SIZE)
    return model

def eff_spoon():
    model = timm.create_model("tf_efficientnet_b3_ns", pretrained=False)
    in_features = model.classifier.in_features
    model.classifier = nn.Linear(in_features=in_features, out_features=TARGET_SIZE)
    return model
    
class Resnext50(nn.Module):
    def __init__(self, model_name='resnext50_32x4d', pretrained=False):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained)
        n_features = self.model.fc.in_features
        self.model.fc = nn.Linear(n_features, TARGET_SIZE)

    def forward(self, x):
        x = self.model(x)
        return x

class EffNetb3(torch.nn.Module):
    def __init__(self):
        super(EffNetb3, self).__init__()
        self.model = timm.create_model("tf_efficientnet_b3_ns", pretrained=False)
        n_features = self.model.classifier.in_features
        self.model.classifier = nn.Sequential(
            nn.Linear(n_features, 1000),
            nn.Dropout(0.2),
            nn.Linear(1000, TARGET_SIZE)
        )
        
    def forward(self, x):
        x = self.model(x)
        return x

In [ ]:
def get_transforms_vanilla():
    return alb.Compose([
        alb.Resize(height=IMG_SIZE[0], width=IMG_SIZE[0]),
        alb.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
        ToTensorV2(p=1.0),
    ], p=1.)

def get_transforms_effnet():
    return alb.Compose([
        alb.CenterCrop(IMG_SIZE[0], IMG_SIZE[0], p=1.),
        alb.Resize(height=IMG_SIZE[0], width=IMG_SIZE[0]),
        alb.Transpose(p=0.5),
        alb.HorizontalFlip(p=0.5),
        alb.VerticalFlip(p=0.5),
        alb.ShiftScaleRotate(p=0.5),
        alb.HueSaturationValue(hue_shift_limit=0.3, sat_shift_limit=0.3, val_shift_limit=0.3, p=0.6),
        alb.RandomBrightnessContrast(brightness_limit=(-0.1,0.2), contrast_limit=(-0.1, 0.2), p=0.5),
        alb.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
        ToTensorV2(p=1.0),
    ], p=1.)

def get_transforms_resnet50():
    return alb.Compose([
        alb.CenterCrop(IMG_SIZE[0], IMG_SIZE[0], p=1.),
        alb.Resize(height=IMG_SIZE[0], width=IMG_SIZE[0]),
        alb.Transpose(p=0.5),
        alb.HorizontalFlip(p=0.5),
        alb.VerticalFlip(p=0.5),
        alb.ShiftScaleRotate(p=0.6),
        alb.HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.3, val_shift_limit=0.3, p=0.4),
        alb.RandomBrightnessContrast(brightness_limit=(-0.1,0.2), contrast_limit=(-0.1, 0.2), p=0.5),
        alb.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
        ToTensorV2(p=1.0),
    ], p=1.)


def get_transforms_spoon_b3():
    return alb.Compose([
            alb.Resize(height=IMG_SIZE[0], width=IMG_SIZE[0]),
            alb.Cutout(num_holes=1, max_h_size=50, max_w_size=50),
            alb.Transpose(p=0.5),
            alb.HorizontalFlip(p=0.5),
            alb.VerticalFlip(p=0.5),
            alb.ShiftScaleRotate(p=0.5),
            alb.HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.3, val_shift_limit=0.3, p=0.5),
            alb.RandomBrightnessContrast(brightness_limit=(-0.1,0.2), contrast_limit=(-0.1, 0.2), p=0.5),
            alb.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
            ToTensor(),
        ], p=1.)

def get_transforms_spoon_r50():
    return alb.Compose([
            alb.Resize(height=IMG_SIZE[0], width=IMG_SIZE[0]),
            alb.Cutout(num_holes=1, max_h_size=50, max_w_size=50),
            alb.Transpose(p=0.5),
            alb.HorizontalFlip(p=0.5),
            alb.VerticalFlip(p=0.5),
            alb.ShiftScaleRotate(p=0.5),
            alb.HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.3, val_shift_limit=0.3, p=0.5),
            alb.RandomBrightnessContrast(brightness_limit=(-0.1,0.2), contrast_limit=(-0.1, 0.2), p=0.5),
            alb.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
            ToTensor(),
        ], p=1.)

In [ ]:
### NO TTA ####
def inference_func(model, data_loader, device):
    """
    Make predictions for different models
    """
    
    model.eval()
    image_preds_all = []
    
    for step, data in enumerate(data_loader):
        x = data["image"].to(device)
        image_preds = model(x)
        image_preds_all += [image_preds.detach().cpu().numpy()]
        
    image_preds_all = np.concatenate(image_preds_all, axis=0)
    return image_preds_all

In [ ]:
def load_model_params(model, model_path, model_list, fold, contrib):
    if contrib == "kjs":
        model.load_state_dict(torch.load(model_path + model_list[fold]))
    if contrib == "spoon":
        model_back = model_list[0].split("-")[0]
        if model_back == "tf_efficientnet_b3_ns":
            model.load_state_dict(torch.load(model_path + model_list[fold]))
        else:
            state_dict = torch.load(model_path + model_list[fold])['model']
            state_dict = {k[13:] if k.startswith('module.model.') else k: state_dict[k] for k in state_dict.keys()}  
            model.load_state_dict(state_dict)
            
    return model

In [ ]:
def MMI(model_p, device, model_path, transforms, model_name, fold_used, model_list, contrib):
    """
    Input : *args \n
    Ouput : N-fold predictions
    """
    
    print("Making prediction for %s" % (model_name))
    
    model_p.to(device)
    test_preds = []
    
    for fold in range(len(fold_used)):
        test_dataset = Cassava_dataset(sample_sub,
                                       transforms=transforms,
                                       img_size=IMG_SIZE,
                                       img_path=TEST_PATH,
                                       is_train=False)

        test_DataLoader = torch.utils.data.DataLoader(test_dataset,
                                                      BATCH_SIZE,
                                                      drop_last=False,
                                                      num_workers=8,
                                                      pin_memory=True,
                                                      shuffle=False)
        
        model_p = load_model_params(model=model_p,
                                    model_path=model_path,
                                    model_list=model_list,
                                    fold=fold,
                                    contrib=contrib)
        
            
        model_p.eval()
        print("Making Inference %s" % (model_list[fold]))
        with torch.no_grad():
            for i in range(TTA):
                print("TTA step : %d" % i)
                test_preds += [inference_func(model_p, test_DataLoader, device)]
     
    # Garbage Collection Function
    del model_p, test_DataLoader
    gc.collect()
    torch.cuda.empty_cache()
    
    return np.mean(test_preds, axis=0)

In [ ]:
# Effnetb1
model_list = ["effnet_b1_512_seed_5_fold_%d.pth" % (i + 1) for i in range(len(EPOCHS_USED))]
model = EffNetb1()
b1_k = MMI(model_p=model,
          device=device,
          model_path="../input/cassava-effnet-5fold-full/",
          transforms=get_transforms_effnet(),
          model_name="Effnet-b1",
          fold_used=EPOCHS_USED,
          model_list=model_list,
          contrib="kjs")

print("\n")
print("PREDS : ", b1_k[0])

In [ ]:
# Effnetb1
model_list = ["resnext50_32x4d-image512-fold%d_5.pth" % (i) for i in range(len(EPOCHS_USED))]
model = Resnext50()
r50_k = MMI(model_p=model,
          device=device,
          model_path="../input/spoon-kjs-cassava-model-weights/best_kjs_cv/",
          transforms=get_transforms_resnet50(),
          model_name="resnext50_32x4d",
          fold_used=EPOCHS_USED,
          model_list=model_list,
          contrib="kjs") 

print("\n")
print("PREDS : ", r50_k[0])

In [ ]:
model_list = ["resnext50_32x4d-image512-fold%d_5.pth" % (i) for i in range(len(EPOCHS_USED))]
model = res_spoon()
r50_p = MMI(model_p=model,
          device=device,
          model_path="../input/spoon-kjs-cassava-model-weights/best_spoon_cv/",
          transforms=get_transforms_vanilla(),
          model_name="resnext50_32x4d",
          fold_used=EPOCHS_USED,
          model_list=model_list ,
          contrib="spoon")

print("\n")
print("PREDS : ", r50_p[0])

In [ ]:
model_list = ["tf_efficientnet_b3_ns-image512-fold%d_5.pth" % (i) for i in range(len(EPOCHS_USED))]
model = eff_spoon()
b3_p = MMI(model_p=model,
          device=device,
          model_path="../input/spoon-kjs-cassava-model-weights/best_spoon_cv/",
          transforms=get_transforms_spoon_b3(),
          model_name="tf_efficientnet_b3_ns",
          fold_used=EPOCHS_USED,
          model_list=model_list ,
          contrib="spoon")

print("\n")
print("PREDS : ", b3_p[0])

In [ ]:
test_sub = sample_sub.copy()
ensemble = (
            (b1_k  * 0.25) +
            (r50_k * 0.15) + 
            (b3_p  * 0.25) +
            (r50_p * 0.35)
           )

print(ensemble[0])
test_sub["label"] = np.argmax(ensemble, axis=1)

In [ ]:
test_sub.to_csv("submission.csv", index=False)
print("KERNEL RUN COMPLETED")
test_sub.head()